<a href="https://colab.research.google.com/github/TurkuNLP/Text_Mining_Course/blob/master/model_explainability.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Model explainability example

* This notebook is based off the BERT text classifier notebook in this same course (https://github.com/TurkuNLP/Text_Mining_Course/blob/master/train_bert_for_text_classification.ipynb)
* Remember to watch the explain-ml tutorial https://explainml-tutorial.github.io/neurips20 this is not meant to be a self-contained notebook!
* The new code related to model explainability starts half-way through, after the classifier has been trained


# Train BERT text classifier using the Transformers library

This notebook shows how to train a simple text classifier by fine-tuning a pre-trained BERT model using the Hugging Face [Transformers](https://huggingface.co/transformers/) library.

This notebook is based in part on the [Text classification on GLUE](https://colab.research.google.com/github/huggingface/notebooks/blob/master/examples/text_classification.ipynb) notebook.

**NOTE**: it's recommended to run this using a runtime with a GPU. Select "Runtime" -> "Change runtime type" from the top menu in Colab and set "Hardware accelerator" to "GPU" when starting

## Install libraries

First, we'll use [`pip`](https://pypi.org/project/pip/) to install two Python libraries that are used in this notebook: [`transformers`](https://huggingface.co/transformers/) and [`datasets`](https://huggingface.co/docs/datasets/).

In [1]:
!pip --quiet install transformers
!pip --quiet install datasets

     |████████████████████████████████| 2.1MB 7.9MB/s 
     |████████████████████████████████| 3.3MB 54.0MB/s 
     |████████████████████████████████| 901kB 51.8MB/s 
     |████████████████████████████████| 204kB 8.1MB/s 
     |████████████████████████████████| 245kB 13.9MB/s 
     |████████████████████████████████| 112kB 16.0MB/s 


## Import libraries

Next, let's import the classes and functions we'll be using (click links for documentation):

* [AutoTokenizer](https://huggingface.co/transformers/model_doc/auto.html#transformers.AutoTokenizer): tokenizer class with support for the tokenization conventions of various pre-trained models
* [AutoModelForSequenceClassification](https://huggingface.co/transformers/model_doc/auto.html#automodelforsequenceclassification): an [AutoModel](https://huggingface.co/transformers/model_doc/auto.html#transformers.AutoModel) class that can load various pre-trained models and supports fine-tuning for text (sequence) classification
* [TrainingArguments](https://huggingface.co/transformers/main_classes/trainer.html#transformers.TrainingArguments): simple class for holding hyperparameters and other settings for model training
* [Trainer](https://huggingface.co/transformers/main_classes/trainer.html): class supporting various forms of training transformer models
* [load_dataset](https://huggingface.co/docs/datasets/package_reference/loading_methods.html#datasets.load_dataset): function for loading datasets e.g. from the [datasets](https://huggingface.co/datasets) collection

In [2]:
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments
from transformers import Trainer
from datasets import load_dataset

## Set model, dataset and hyperparameters

Let's then set some global variables such as the name of the pre-trained model and the hyperparameters we'll use for fine-tuning it.

* `MODEL_NAME`: the name of a pretrained model included in the [model repository](https://huggingface.co/models)
* `DATASET`: the path and name of a dataset included in the [dataset repository](https://huggingface.co/datasets)
* `LEARNING_RATE`, `BATCH_SIZE`, and `TRAIN_EPOCHS`: hyperparameters to use for fine-turning the model. (Try different values here!)

Here, we'll use the [Stanford Sentiment Treebank](https://huggingface.co/datasets/sst) dataset as prepared for the [GLUE](https://huggingface.co/datasets/glue) collection. This is a binary text classification task where the objective is to determine if sentences express a positive or negative sentiment.

In [3]:
MODEL_NAME = 'bert-base-cased'
DATASET = ('glue', 'sst2')

## Load dataset

We'll first load the dataset with [`load_dataset`](https://huggingface.co/docs/datasets/package_reference/loading_methods.html#datasets.load_dataset) and determine the number of distinct labels in the (training) data.

In [7]:
dataset = load_dataset(*DATASET)
num_labels = len(set(dataset['train']['label']))

Reusing dataset glue (/root/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


The loaded dataset is a simple dictionary-like container for distinct `DataSet` objects for training, development (validation), and test data:

In [8]:
print(dataset)
print(f'number of distinct labels: {num_labels}')

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 67349
    })
    validation: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 872
    })
    test: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 1821
    })
})
number of distinct labels: 2


To reduce training time a bit, let's just take every 10th item in the `train` subset of the dataset.

In [6]:
# I leave this undone to get the best possible model later for the explainability part
dataset['train'] = dataset['train'].filter(lambda example, idx: idx % 10 == 0, with_indices=True)

## Load tokenizer and tokenize data

We'll then load an appropriate tokenizer for the pre-trained model we'll be using with [`AutoTokenizer.from_pretrained`](https://huggingface.co/transformers/model_doc/auto.html#transformers.AutoTokenizer.from_pretrained)

In [9]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

The tokenizer can take a string (or e.g. a list of strings) and performs all the necessary preprocessing steps to prepare its input for use by the model, including splitting input texts into tokens, adding padding and special characters, and mapping those to integer IDs (`input_ids`). The tokenizer also creates attention weights (`attention_mask`) for the model.

In [10]:
tokenizer('Hello, BERT!')    # Demonstrate tokenizer output

{'input_ids': [101, 8667, 117, 139, 9637, 1942, 106, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]}

We'll then define an encoding function applying the tokenizer to the text data  (key `"sentence"`) of a [`Dataset`](https://huggingface.co/docs/datasets/package_reference/main_classes.html?highlight=dataset#datasets.Dataset) object, and use the `map` function of the `DatasetDict` to tokenize the train, development, and test datasets.

In [11]:
def encode_dataset(d):
  return tokenizer(d['sentence'])

encoded_dataset = dataset.map(encode_dataset)

## Load pre-trained model

Next, we'll load the pre-trained model with support for text classification output using [`AutoModelForSequenceClassification.from_prertained`](https://huggingface.co/transformers/model_doc/auto.html#transformers.AutoModelForSequenceClassification.from_pretrained).

Note that we need to provide the number of labels in the data when loading the model.

In [12]:
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=num_labels)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

## Training parameters and metrics
We're almost ready to train. We'll next create a [`TrainingArguments`](https://huggingface.co/transformers/main_classes/trainer.html#transformers.TrainingArguments) object to hold the hyperparameters and other settings that are appropriate for training on Colab:

* `save_strategy`: set to `"epoch"` so that model checkpoints are saved every epoch, this I can do because I save to own Google Drive
* `evaluation_strategy` and `logging_strategy` set to `"epoch"` so that evaluation and logging are performed once per epoch
* The hyperparameters `LEARNING_RATE`, `BATCH_SIZE` and `TRAIN_EPOCHS` set above are passed to the training process through this object

Finally, we'll define a simple accuracy metric measuring how many predictions match their correct values.

In [13]:
def compute_accuracy(pred):
    y_pred = pred.predictions.argmax(axis=1)
    y_true = pred.label_ids
    return { 'accuracy': sum(y_pred == y_true) / len(y_true) }

# Training (fine-tuning)

For fine-tuning the pre-trained model, we'll create a [`Trainer`](https://huggingface.co/transformers/main_classes/trainer.html) object, providing it with the pre-trained model, settings, training and development (validation) data, and the evaluation metric created above.

In [27]:


# Hyperparameters
LEARNING_RATE=1e-4
BATCH_SIZE=128
TRAIN_EPOCHS=4

In [28]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [29]:
train_args = TrainingArguments(
    '/content/drive/MyDrive/WorkStuff/ssd2-sentiment-model',    # output directory for checkpoints and predictions
    load_best_model_at_end=True,
    evaluation_strategy='epoch',
    logging_strategy='epoch',
    learning_rate=LEARNING_RATE,
    per_device_train_batch_size=BATCH_SIZE,
    num_train_epochs=TRAIN_EPOCHS,
)

In [30]:
trainer = Trainer(
      model,
      train_args,
      train_dataset=encoded_dataset['train'],
      eval_dataset=encoded_dataset['validation'],
      tokenizer=tokenizer,
      compute_metrics=compute_accuracy
)

Training is then performed simply by calling the `train` function of the `Trainer` object.

In [31]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Runtime,Samples Per Second
1,0.224500,0.251691,0.901376,3.350500,260.260000
2,0.112600,0.265011,0.903670,3.331200,261.767000
3,0.064200,0.299133,0.905963,3.359100,259.593000
4,0.031400,0.303404,0.915138,3.323100,262.409000


TrainOutput(global_step=2108, training_loss=0.10818358209599127, metrics={'train_runtime': 2452.1031, 'train_samples_per_second': 0.86, 'total_flos': 9023432196007320.0, 'epoch': 4.0, 'init_mem_cpu_alloc_delta': 0, 'init_mem_gpu_alloc_delta': 433776128, 'init_mem_cpu_peaked_delta': 0, 'init_mem_gpu_peaked_delta': 0, 'train_mem_cpu_alloc_delta': 5898240, 'train_mem_gpu_alloc_delta': 1741112320, 'train_mem_cpu_peaked_delta': 0, 'train_mem_gpu_peaked_delta': 6627404800})

## Evaluation and predictions for user input

We can use `trainer.evaluate()` to evaluate the trained model on the `eval_dataset` given to the trainer:

In [20]:
results = trainer.evaluate()
print(f'Accuracy: {results["eval_accuracy"]}')

Accuracy: 0.9162844036697247


# Model save

* Now that the model is trained, we may want to save it to Google Drive
* That way we don't need to train it every time

In [32]:
import torch
#make sure you make this directory
torch.save(trainer.model,"/content/drive/MyDrive/WorkStuff/sent_model.pt")

# Model load

* Now we load the model and repeat some of the imports so it is possible to run the notebook from this point onwards

In [33]:
# relevant stuff repeated from above so you can run it from here onwards if you happen to have the model trained

from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments
from transformers import Trainer
from datasets import load_dataset

MODEL_NAME = 'bert-base-cased'
DATASET = ('glue', 'sst2')

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

In [23]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [34]:
import torch
model = torch.load("/content/drive/MyDrive/WorkStuff/sent_model.pt")
model.to('cpu')    # simplifies input placement

sentiment_label = [
    'negative',
    'positive'
]

def predict_sentiment(string):
    tokenized = tokenizer(string, return_tensors='pt')
    pred = model(**tokenized)
    pred_idx = pred.logits.detach().numpy().argmax()
    return sentiment_label[pred_idx]

try that out

In [35]:
example_sentences = [
    'I think this is very good, and I am in support.',
    'This is bad news.',
    'This could not have been better.',
    'This is not good news at all.',
    'Donald Trump lost the election and will no longer be every day on CNN.',
    'I think your service fulfilled my expectations, I could not really expect you to be any better'
]

for e in example_sentences:
    print(e, '->', predict_sentiment(e))

I think this is very good, and I am in support. -> positive
This is bad news. -> negative
This could not have been better. -> positive
This is not good news at all. -> negative
Donald Trump lost the election and will no longer be every day on CNN. -> negative
I think your service fulfilled my expectations, I could not really expect you to be any better -> positive


# Model explainability with the captum library

* captum.ai is a nifty library with many of the explainability algorithms implemented
* we will use it here to try the techniques
* this is not the easiest of code, btw, many of the libraries are bleeding-edge, so you cut yourself here and there
* the code below is (quite substantial) adaptation of https://captum.ai/tutorials/Bert_SQUAD_Interpret

In [36]:
!pip install captum pandas matplotlib seaborn

     |████████████████████████████████| 4.4MB 8.3MB/s 


In [52]:
from captum.attr import visualization as viz
from captum.attr import IntegratedGradients, LayerConductance, LayerIntegratedGradients
from captum.attr import configure_interpretable_embedding_layer, remove_interpretable_embedding_layer

In [53]:
#Tells the model that it is in evaluation mode, and zeroes out the gradients
model.eval()
model.zero_grad()

# Helper functions

In [194]:
# Forward on the model -> data in, prediction out, nothing fancy really
def predict(inputs, token_type_ids=None, position_ids=None, attention_mask=None):
    pred=model(inputs, token_type_ids=token_type_ids,
                 position_ids=position_ids, attention_mask=attention_mask)
    return pred.logits #return the output of the classification layer

ref_token_id = tokenizer.pad_token_id # A token used for generating token reference
sep_token_id = tokenizer.sep_token_id # A token used as a separator between question and text and it is also added to the end of the text.
cls_token_id = tokenizer.cls_token_id # A token used for prepending to the concatenated question-text word sequence


In [93]:
# Given input text, construct a pair of (text input, blank reference input as long as the text itself)
# ...token indices:
def construct_input_ref_pair(text, ref_token_id, sep_token_id, cls_token_id, device):
    text_ids = tokenizer.encode(text, add_special_tokens=False)

    # construct input token ids
    input_ids = [cls_token_id] + text_ids + [sep_token_id] #the standard way of feeding the input in

    # construct reference token ids 
    ref_input_ids = [cls_token_id] + [ref_token_id] * len(text_ids) + [sep_token_id]  #basically [CLS] [PAD] [PAD] [PAD] ... [SEP] ... blank
    return torch.tensor([input_ids], device=device), torch.tensor([ref_input_ids], device=device)

# ...token types - since we only have one sentence, these are always 0
def construct_input_ref_token_type_pair(input_ids, device):
    seq_len = input_ids.size(1)
    token_type_ids = torch.zeros((1,seq_len), dtype=torch.long, device=device)
    ref_token_type_ids = torch.zeros_like(token_type_ids, device=device)
    return token_type_ids, ref_token_type_ids

# ...token positions
def construct_input_ref_pos_id_pair(input_ids, device):
    seq_length = input_ids.size(1)
    position_ids = torch.arange(seq_length, dtype=torch.long, device=device)
    # we could potentially also use random permutation with `torch.randperm(seq_length, device=device)`
    ref_position_ids = torch.zeros(seq_length, dtype=torch.long, device=device)

    #make sure shapes match
    position_ids = position_ids.unsqueeze(0).expand_as(input_ids)
    ref_position_ids = ref_position_ids.unsqueeze(0).expand_as(input_ids)
    return position_ids, ref_position_ids

# ...attention mask, that is the same for both input and reference and basically all ones
def construct_attention_mask(input_ids,device):
    return torch.ones_like(input_ids,device=device)



In [192]:
# Let's try it!

device=model.device

text="This piece of news is great, even if you think it is rather unfortunate"

#input:
input_ids, ref_input_ids = construct_input_ref_pair(text, ref_token_id, sep_token_id, cls_token_id, device)
#token type:
token_type_ids, ref_token_type_ids = construct_input_ref_token_type_pair(input_ids, device)
#position ids:
position_ids, ref_position_ids = construct_input_ref_pos_id_pair(input_ids, device)
#attention mask:
attention_mask = construct_attention_mask(input_ids, device)

all_tokens = tokenizer.convert_ids_to_tokens(input_ids[0])
print(all_tokens)

['[CLS]', 'This', 'piece', 'of', 'news', 'is', 'great', ',', 'even', 'if', 'you', 'think', 'it', 'is', 'rather', 'unfortunate', '[SEP]']


In [195]:
p=predict(input_ids,token_type_ids=token_type_ids,position_ids=position_ids,attention_mask=attention_mask)
print("p=",p)
print("p.shape",p.shape)

p= tensor([[-2.3281,  2.5443]], grad_fn=<AddmmBackward>)
p.shape torch.Size([1, 2])


In [196]:
# Yay, now we finally made it to the attribution part
lig = LayerIntegratedGradients(predict, model.bert.embeddings) #attribute the output wrt to embeddings

In [213]:
# inputs: inputs
# baselines: the blank baseline
# target: which of the two classes in the output (pos/neg) to run the prediction against?
attrs, delta = lig.attribute(inputs=(input_ids,token_type_ids,position_ids,attention_mask),
                                  baselines=(ref_input_ids,ref_token_type_ids,ref_position_ids,attention_mask),
                                  return_convergence_delta=True,target=1)
print("attrs shape",attrs.shape)

attrs shape torch.Size([1, 17, 768])


In [214]:
def summarize_attributions(attributions):
    attributions = attributions.sum(dim=-1).squeeze(0)
    attributions = attributions / torch.norm(attributions)
    return attributions

attrs_sum = summarize_attributions(attrs)
print("attrs_sum shape",attrs_sum.shape)

attrs_sum shape torch.Size([17])


In [215]:
print(attrs_sum)
print(tokenizer.convert_ids_to_tokens(input_ids[0]))

for a,t in zip(attrs_sum,tokenizer.convert_ids_to_tokens(input_ids[0])):
    print(float(a),t)

tensor([ 0.0000, -0.0561, -0.0919, -0.0823, -0.0769, -0.0770,  0.8852, -0.0458,
         0.0270, -0.1094, -0.0727, -0.1350, -0.0979, -0.1441, -0.1881, -0.2807,
         0.0575], dtype=torch.float64)
['[CLS]', 'This', 'piece', 'of', 'news', 'is', 'great', ',', 'even', 'if', 'you', 'think', 'it', 'is', 'rather', 'unfortunate', '[SEP]']
0.0 [CLS]
-0.05606936110561358 This
-0.0918954617673179 piece
-0.082304149964058 of
-0.07693875808018998 news
-0.07697800735507292 is
0.8852405197762391 great
-0.0457627638431411 ,
0.027032084672280212 even
-0.10943371691491163 if
-0.07266291499865889 you
-0.1349669216789558 think
-0.09788820942518903 it
-0.14408741002558845 is
-0.18808658280174176 rather
-0.28074980323111726 unfortunate
0.05748516461738326 [SEP]


Damn, that seems to work!

In [226]:
import captum
from IPython.core.display import HTML, display
x=captum.attr.visualization.format_word_importances(tokenizer.convert_ids_to_tokens(input_ids[0]),attrs_sum)
HTML(x)

# Almost there...

* Let's wrap this all into a function

In [253]:


def predict_and_explain(model,text):
    model.zero_grad() #to be safe perhaps it's not needed
    device=model.device
    #input:
    input_ids, ref_input_ids = construct_input_ref_pair(text, ref_token_id, sep_token_id, cls_token_id, device)
    #token type:
    token_type_ids, ref_token_type_ids = construct_input_ref_token_type_pair(input_ids, device)
    #position ids:
    position_ids, ref_position_ids = construct_input_ref_pos_id_pair(input_ids, device)
    #attention mask:
    attention_mask = construct_attention_mask(input_ids, device)

    all_tokens = tokenizer.convert_ids_to_tokens(input_ids[0])

    lig = LayerIntegratedGradients(predict, model.bert.embeddings)
    prediction=predict(input_ids,token_type_ids,position_ids,attention_mask)[0]
    prediction_cls=int(torch.argmax(prediction))
    print("Prediction:", ("negative","positive")[prediction_cls],"Weights:",prediction.tolist())
    for target,classname in enumerate(("negative","positive")):
        
        attrs, delta = lig.attribute(inputs=(input_ids,token_type_ids,position_ids,attention_mask),
                                  baselines=(ref_input_ids,ref_token_type_ids,ref_position_ids,attention_mask),
                                  return_convergence_delta=True,target=target)
        attrs_sum = summarize_attributions(attrs)

        x=captum.attr.visualization.format_word_importances(all_tokens,attrs_sum)
        print("ATTRIBUTION WITH RESPECT TO",classname)
        display(HTML(x))
        print()

In [261]:
predict_and_explain(model,"I like Filip, who used to hang out in the Turku office but now is back to the university! Hell, *everyone* likes him.")

Prediction: positive Weights: [-1.37552011013031, 1.6477326154708862]
ATTRIBUTION WITH RESPECT TO negative



ATTRIBUTION WITH RESPECT TO positive
